In [181]:
import numpy as np
import pandas as pd
from datetime import date
import pickle as pkl
import os
import seaborn as sn
import matplotlib.pyplot as plt


In [182]:
ingredients_recipes = {}
author_recipes = {}

In [183]:
def read_file(path):
    file = pd.read_csv(path)
    df = pd.DataFrame(file)
    return df

In [184]:
def create_ingredients_recipes(df):
    for i in range(len(df)):
        ingredients = df["Ingredients"][i]
        list_ingred = ingredients.split(',')
        for ing in list_ingred:
            if ing not in ingredients_recipes:
                ingredients_recipes[ing] = [df["Recipe Name"][i]]
            else:
                ingredients_recipes[ing].append(df["Recipe Name"][i])


In [185]:
def create_author_recipes(df):
    for i in range(len(df)):
        author = df["Author"][i]
        if author not in author_recipes:
            author_recipes[author] = 1
        else:
            author_recipes[author] += 1

In [186]:
def plot_distributions():
    x_values1 = [ingredients for ingredients in ingredients_recipes if len(ingredients_recipes[ingredients]) > 100]
    y_values1 = [len(ingredients_recipes[ingredients]) for ingredients in ingredients_recipes if len(ingredients_recipes[ingredients]) > 100]
    plt.figure(figsize=(15, 5))
    plt.bar(x_values1, y_values1, color='blue')                
    plt.title("Ingredients distributions")
    plt.xticks(rotation='vertical', fontsize=5)
    plt.xlabel("Ingredients")
    plt.ylabel("Number of recipes")
    plt.show()

    # if len(author_recipes[author_recipes]) > 100
    x_values2 = [author for author in author_recipes]
    y_values2 = [author_recipes[author] for author in author_recipes]
    plt.figure(figsize=(10, 20))
    plt.barh(x_values2, y_values2, color='blue')                
    plt.title("Author distributions")
    plt.yticks(fontsize=2)
    plt.xlabel("Author")
    plt.ylabel("Number of recipes")
    plt.show()

In [187]:
def user_interface():
    print("-"*40 + " Welcome to Pendia " + "-"*40)
    print("1- Log In;\n2- Sing In;\n0- Leave;")
    option = input("1- Log In\n2- Sing In\n0- Leave\n")
    print(option)
    name = ""
    password = ""
    while option != "0":
        state = False
        if os.path.exists("users.pkl"):
            with open("users.pkl", 'rb') as arq:
                d_users =  pkl.load(arq)
        else:
            d_users = {}
        if option == "1" and d_users == {}:
            print("Without users")
        elif option == "1":
            while(1):
                name = input("Name: ")
                print("Name: " + name)
                if name in d_users:
                    break
                else:
                    print("Incorret Name")
            while(1):
                password = input("Password: ")
                print("Password: " + password)
                if d_users[name] == password:
                    break
                else:
                    print("Incorret Password")
            print("-"*40 + " Hi " + name + " " + "-"*40)
            state = True
        elif option == "2":
            while(1):
                name = input("Name: ")
                print("Name: " + name)
                if name not in d_users:
                    password = input("Password: ")
                    print("Password: " + password)
                    d_users[name] = password
                    break
                else:
                    print("User Already exists")
            with open("users.pkl", 'wb') as arq:
                pkl.dump(d_users, arq)
            print("-"*40 + " Hi " + name + " " + "-"*40)
            state = True
        else:
            print("Invalid Option")
        
        if state:
            option = "0"
        else:
            print("1- Log In;\n2- Sing In;\n0- Leave;")
            option = input("1- Log In;\n2- Sing In;\n0- Leave;\n")
            print(option)

    path = "dataset/" + name + ".csv"
    df = read_file(path)
    display(df)
    

        
    


In [188]:
def main():
    path = "dataset/clean_recipes.csv"
    df = read_file(path)
    create_ingredients_recipes(df)
    create_author_recipes(df)
    #plot_distributions()    
    user_interface()

    
    

In [189]:
if __name__ == '__main__':
    main()

---------------------------------------- Welcome to Pendia ----------------------------------------
1- Log In;
2- Sing In;
0- Leave;
1
Name: Carlos
Password: 1234
---------------------------------------- Hi Carlos ----------------------------------------


,Ingredients,Quantity,Expiration Date
0,flour,1,26/06/2025
1,flour,3,26/01/2024
2,egg,16,29/12/2023
3,milk,2,26/12/2023
4,margarine,1,9/02/2024
5,salt,2,29/11/2025
6,white sugar,1,29/05/2024
7,bread,5,15/12/2023
8,vegetable oil,3,15/06/2025
9,onion,9,19/12/2023
